In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import pandas as pd
import numpy as np
import requests
from time import sleep
from csv import writer

In [3]:
data = pd.read_excel(r'C:\Users\Anmo\Desktop\notebooks\Klinikliste.xlsx')
df = pd.DataFrame(data, columns= ['Klinikname','Link Google Maps','Link Klinikbewertungen'])
df

,Klinikname,Link Google Maps,Link Klinikbewertungen
0,Augenklinik Dr.Hoffmann,https://www.google.com/maps/place/Augenklinik+...,https://www.klinikbewertungen.de/klinik-forum/...
1,Krankenhaus Marienstift Braunschweig,https://www.google.com/maps/place/Krankenhaus+...,https://www.klinikbewertungen.de/klinik-forum/...
2,Herzogin Elisabeth Hospital Braunschweig,https://www.google.com/maps/place/Herzogin+Eli...,https://www.klinikbewertungen.de/klinik-forum/...
3,Asklepios Fachklinikum Göttingen,https://www.google.com/maps/place/Asklepios+Fa...,https://www.klinikbewertungen.de/klinik-forum/...
4,Asklepios Fachklinikum Tiefenbrunn,https://www.google.com/maps/place/Asklepios+Fa...,https://www.klinikbewertungen.de/klinik-forum/...
5,Friederikenstift Hannover,https://www.google.com/maps/place/DIAKOVERE+Fr...,https://www.klinikbewertungen.de/klinik-forum/...
6,Annastift Hannover,https://www.google.com/maps/place/DIAKOVERE+An...,https://www.klinikbewertungen.de/klinik-forum/...
7,Clementinenhaus,https://www.google.com/maps/place/DRK-Krankenh...,https://www.klinikbewertungen.de/klinik-forum/...
8,Sophienklinik Hannover,https://www.google.com/maps/place/Sophienklini...,https://www.klinikbewertungen.de/klinik-forum/...
9,KRH Klinikum Großburgwedel,https://www.google.com/maps/place/KRH+Klinikum...,https://www.klinikbewertungen.de/klinik-forum/...


In [4]:
website_total = df.shape
total_websites=(website_total[0])
print('TOTAL WEBSITES: ',total_websites)

TOTAL WEBSITES:  14


In [13]:
#looping through the klinikliste
for u in range(df.shape[0]+1):
    url =df.iloc[u,1]


    print ('\n\nWebsites found:', u+1, ' of ',total_websites,'')

    # chrome path location
    chrome_driver_path = "C:/Users/Anmo/anaconda3/chromedriver.exe"
    chrome_options = Options()

     #using headless means it wont display the driver while its working
    chrome_options.add_argument("--headless")

    driver = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)
    #we use the sleep so we can limit the working time so we dont get banned
    sleep(10)
    driver.get(url)
    sleep(5)
    driver.maximize_window()
    wait = WebDriverWait(driver,10)

    # diplaying the Title
    klinik_name = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
    print(klinik_name)


    # expanding the whole Reviews by clicking the reviews btn
    reviews_number = 0
    all_reviews = driver.find_element_by_class_name('widget-pane-link')
    reviews_number = all_reviews.text.split()
    reviews_number = reviews_number[0].strip()
    reviews_number = int(reviews_number)
    all_reviews.click()
    
    sleep(10)
    
    
    # count and scrolling
    def scroll_review():
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "section-layout-root")))
        #reviews_numb = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/button').get_attribute('aria-label')
        scrolls_number = (reviews_number//10)+1
        print("Maximum scroll needed to get all Reviews:" , scrolls_number)
        sleep(6)

        x=0
        while (x < scrolls_number):
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "section-layout-root")))
            try:
                scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show') # It gets the section of the scroll bar.
                print( '  ',x+1, '  times scroll')
                sleep(5)
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            except:
                pass
            sleep(5)    
            x=x+1
             
    sleep(5)
    scroll_review()



    # getting the full reviews
    def expand_reviews():
        expand = driver.find_elements_by_class_name("section-expand-review")

        for i in expand:
            i.click()

    #repressing it
    print('\nExpanding Review...')
    sleep(2)
    expand_reviews()



    # Getting the full review text
    def text_reviews():
        t_reviews = driver.find_elements_by_class_name("section-review-review-content")
        txt_review=[]

        
        for i in t_reviews:
            txt_review.append(i.text.strip())
        return(txt_review)



    # Counting the number of reviews
    t_reviews = driver.find_elements_by_class_name("section-review-review-content")
    print('Total number of reviews are ',len(t_reviews))


    # Getting the star rating
    def star_reviews():
        s_reviews = driver.find_elements_by_css_selector("[class='section-review-stars']")
        
        star_reviews_list = []
        
        for a in s_reviews:
            star_reviews_list.append(a.get_attribute("aria-label"))
        return(star_reviews_list)
        

    # RECALLNG STAR REVIEWS FUNTION
    star_reviews()



    # DATE OF REVIEWS SCRAPING - FUNCTION

    def date_reviews():
        date_reviews = driver.find_elements_by_css_selector("[class='section-review-publish-date']")
        datum_list =[]
        for i in date_reviews:
            datum_list.append(i.text)
        
        return(datum_list)




    # NUMBER OF LIKES SCRAPING - FUNCTION
    def total_likes():
        
        t_reviews = driver.find_elements_by_class_name("section-review-review-content")                                                        # Retreiving TOTAL NUMBER OF REVIEWS including without Likes button option
        t_like = driver.find_elements_by_class_name("section-review-interactions-label")                                                           # Retreiving total number of LIKES
        likes_list=[]
        for i in t_reviews:
            likes_list.append(0)

        Likes_len = (len(likes_list))                                                    
        
        number=0
        while (number < Likes_len) :
        
            for l in t_like:
                lk=(l.text).strip()
                if lk != 'Share':
                    if lk =='Like' or '':
                        lk = 0
                    else:
                        lk=lk

                    likes_list[number]=lk
                    number+=1
            
            return(likes_list)



    # WRITING IN CSV - EXCEL FORM

    print('\nAdding data in a CSV file...\n')

    df_klinik = pd.DataFrame(
        {'Name der Klinik': klinik_name,
        'Sternbewertung':star_reviews(), 
        'Datum der Bewertung': date_reviews(),
        'Likes': total_likes(),
        'Textuelle Bewertung': text_reviews()
        })


    datei='C:/Users/Anmo/Desktop/notebooks/scrapted_data.csv'
    df_klinik.to_csv(datei, mode='a', index=False, header=True, encoding="utf-8")

driver.close()




Websites found: 1  of  14 
Augenklinik Dr. Hoffmann
Maximum scroll needed to get all Reviews: 11
   1   times scroll
   2   times scroll
   3   times scroll
   4   times scroll
   5   times scroll
   6   times scroll
   7   times scroll
   8   times scroll
   9   times scroll
   10   times scroll
   11   times scroll

Expanding Review...
Total number of reviews are  108

Adding data in a CSV file...



Websites found: 2  of  14 
Krankenhaus Marienstift gGmbH
Maximum scroll needed to get all Reviews: 9
   1   times scroll
   2   times scroll
   3   times scroll
   4   times scroll
   5   times scroll
   6   times scroll
   7   times scroll
   8   times scroll
   9   times scroll

Expanding Review...
Total number of reviews are  86

Adding data in a CSV file...



Websites found: 3  of  14 
Herzogin Elisabeth Hospital
Maximum scroll needed to get all Reviews: 14
   1   times scroll
   2   times scroll
   3   times scroll
   4   times scroll
   5   times scroll
   6   times scroll
   7 

IndexError: single positional indexer is out-of-bounds